In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(2)


In [3]:
FRAC_ADMIT = df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

A    0.369469
dtype: float64

In [5]:
len(df)

1000000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision,B_p
261180,0,12,0.03,9,12,-3.825614,-3.372248,0,0
275905,0,11,0.03,8,11,-3.904445,-3.451079,0,0
601095,0,10,0.03,11,10,-3.667952,-3.529910,0,0
32461,0,12,0.03,8,12,-3.904445,-3.372248,0,0
489547,0,12,0.03,10,12,-3.746783,-3.372248,0,0
...,...,...,...,...,...,...,...,...,...
897432,1,106,1.23,106,121,3.820987,5.220324,1,1
992959,1,107,1.23,107,124,3.899818,5.456817,1,1
934719,1,107,1.23,107,124,3.899818,5.456817,1,1
454722,1,105,1.23,105,120,3.742157,5.141493,1,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','ml_outcomes']].groupby(['R','T','B_p']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p']].groupby(['R','T','B_p']).count().reset_index()
df_count.columns = ['R','T','B_p','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,ml_outcomes,N
1,397,1,0,11,0,0.03,1
5,396,5,0,14,1,0.04,1
7,395,7,0,15,1,0.08,2
0,394,0,0,10,0,0.09,3
10,393,10,0,17,1,0.15,3
...,...,...,...,...,...,...,...
106,4,106,0,65,1,6637.80,9620
98,3,98,0,61,1,6673.68,10764
104,2,104,0,64,1,6750.92,10076
102,1,102,0,63,1,6752.46,10231


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,B_p,ml_outcomes,N
0,394,0,0,10,0,0.09,3
1,397,1,0,11,0,0.03,1
2,391,2,0,12,0,0.21,7
3,384,3,0,13,0,0.48,12
4,385,4,0,14,0,0.44,11
...,...,...,...,...,...,...,...
393,335,393,1,105,1,3.69,3
394,363,394,1,106,1,1.23,1
395,346,395,1,107,1,2.46,2
396,362,396,1,108,1,1.23,1


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

369469


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'])]
    key2 = (row['R'],row['T'], 1-row['B_p'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(46, 352)

In [21]:
white_pass_boards = []
white_fail_boards = []
black_pass_boards = []
black_fail_boards = []

for key in vars_cache:
    r, t, b_p = key
    if b_p == 1 and r==0:
        white_pass_boards.append(key)
    elif b_p == 0 and r==0:
        white_fail_boards.append(key)
    elif b_p == 1 and r==1:
        black_pass_boards.append(key)
    elif b_p == 0 and r==1:
        black_fail_boards.append(key)

len(white_pass_boards),len(white_fail_boards),len(black_pass_boards),len(black_fail_boards)

(114, 98, 96, 90)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p']].groupby(['R','B_p']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

399

In [24]:
#Of those who pass the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in white_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in white_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,1.0
4,4,0.0
...,...,...
393,393,1.0
394,394,1.0
395,395,1.0
396,396,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
100,0,100,0,62,1,6753.28,10552,100,1.0
102,1,102,0,63,1,6752.46,10231,102,1.0
104,2,104,0,64,1,6750.92,10076,104,1.0
98,3,98,0,61,1,6673.68,10764,98,1.0
106,4,106,0,65,1,6637.80,9620,106,1.0
...,...,...,...,...,...,...,...,...,...
10,393,10,0,17,1,0.15,3,10,0.0
0,394,0,0,10,0,0.09,3,0,0.0
7,395,7,0,15,1,0.08,2,7,0.0
5,396,5,0,14,1,0.04,1,5,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
212,388,212,1,6,0,0.27,1,212,0.0
213,389,213,1,7,0,0.27,1,213,0.0
214,383,214,1,8,0,0.54,2,214,0.0
215,378,215,1,9,0,0.81,3,215,0.0
0,394,0,0,10,0,0.09,3,0,0.0
...,...,...,...,...,...,...,...,...,...
207,370,207,0,124,1,1.00,1,207,1.0
208,368,208,0,125,1,1.00,1,208,1.0
209,371,209,0,127,1,1.00,1,209,1.0
210,372,210,0,129,1,1.00,1,210,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
100,0,100,0,62,1,6753.28,10552,100,1.0
102,1,102,0,63,1,6752.46,10231,102,1.0
104,2,104,0,64,1,6750.92,10076,104,1.0
98,3,98,0,61,1,6673.68,10764,98,1.0
106,4,106,0,65,1,6637.80,9620,106,1.0
...,...,...,...,...,...,...,...,...,...
10,393,10,0,17,1,0.15,3,10,0.0
0,394,0,0,10,0,0.09,3,0,0.0
7,395,7,0,15,1,0.08,2,7,0.0
5,396,5,0,14,1,0.04,1,5,0.0


In [35]:
xxx.to_csv('./decision.csv')

In [36]:
solution = pd.read_csv('./decision.csv')

In [37]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
100,0,100,0,62,1,6753.28,10552,100,1.0
102,1,102,0,63,1,6752.46,10231,102,1.0
104,2,104,0,64,1,6750.92,10076,104,1.0
98,3,98,0,61,1,6673.68,10764,98,1.0
106,4,106,0,65,1,6637.80,9620,106,1.0
...,...,...,...,...,...,...,...,...,...
10,393,10,0,17,1,0.15,3,10,0.0
0,394,0,0,10,0,0.09,3,0,0.0
7,395,7,0,15,1,0.08,2,7,0.0
5,396,5,0,14,1,0.04,1,5,0.0


In [38]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)

In [39]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
100,0,100,0,62,1,6753.28,10552,100,1.0,0_62_1
102,1,102,0,63,1,6752.46,10231,102,1.0,0_63_1
104,2,104,0,64,1,6750.92,10076,104,1.0,0_64_1
98,3,98,0,61,1,6673.68,10764,98,1.0,0_61_1
106,4,106,0,65,1,6637.80,9620,106,1.0,0_65_1
...,...,...,...,...,...,...,...,...,...,...
10,393,10,0,17,1,0.15,3,10,0.0,0_17_1
0,394,0,0,10,0,0.09,3,0,0.0,0_10_0
7,395,7,0,15,1,0.08,2,7,0.0,0_15_1
5,396,5,0,14,1,0.04,1,5,0.0,0_14_1


In [40]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT['A']

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [41]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [42]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
1,397,1,0,11,0,0.03,1,1,0.0,0_11_0
43,165,43,0,34,0,685.12,4282,43,0.0,0_34_0
58,157,58,0,41,1,841.00,3364,58,0.0,0_41_1
45,156,45,0,35,0,842.69,4957,45,0.0,0_35_0
60,145,60,0,42,1,1035.72,3836,60,0.0,0_42_1
...,...,...,...,...,...,...,...,...,...,...
129,176,129,0,77,0,545.70,642,129,1.0,0_77_0
160,179,160,0,92,1,520.60,548,160,1.0,0_92_1
131,183,131,0,78,0,452.36,526,131,1.0,0_78_0
133,189,133,0,79,0,391.50,450,133,1.0,0_79_0


In [43]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
281,29,281,1,44,0,4826.13,9463,281,0.0,1_44_0
250,247,250,1,28,1,70.72,208,250,0.0,1_28_1
252,238,252,1,29,1,104.65,299,252,0.0,1_29_1
235,235,235,1,21,0,112.20,374,235,0.0,1_21_0
254,232,254,1,30,1,121.80,348,254,0.0,1_30_1
...,...,...,...,...,...,...,...,...,...,...
356,211,356,1,81,1,225.33,203,356,1.0,1_81_1
337,210,337,1,72,0,229.27,227,337,1.0,1_72_0
354,205,354,1,80,1,262.90,239,354,1.0,1_80_1
312,90,312,1,59,1,2407.86,3087,312,1.0,1_59_1


In [44]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.36946900000000016